# PX915 Individual Project Reproducible Result - Ben Gosling

### Import Modules

In [ ]:
%matplotlib inline
# import epoch calculators
from multiprocessing import Pool
import numpy as np
import sdf
from scipy import constants

from Py_scripts.run_epoch import * # stores functions which aid in the running of epoch simulations
from Py_scripts.sim_setup import * # stores functions which aid in the creation of setting up epoch runs

# Gaussian process regression scripts
from Py_scripts.gp import * # stores functions for performing GP regression for 1D input space
from Py_scripts.utils import read_json_file

### Constants

In [ ]:
mu0 = constants.mu_0 # permeability of free space
pi = np.pi # pi
pico = 1e-12 # pico prefix (operating at pico-second time scale)
micron = 1e-6 # micro prefix (operating at many microns for length scale)

# EPOCH simulation set-up
EPOCH requires you to specify an output directory which stores the input file to set up the simulation and store the output files. The python function below is used to create a directory within epoch_surra and populate it with one of the example input decks in the input_decks directory. 

In [ ]:
# set name of the output directory
dir = 'Data_epoch'
sub_dirs = [f'Data_{i}' for i in range(1, 11)]
dirs = [f'Data_epoch/Data_{i}' for i in range(1,11)]
# input file/setup used throughout the report
input_file = 'example_input.deck'
# set initial laser intensity in W/cm^2 (varies between 1e14 - 1e16 in the report)
intensity = 3e15 # set initial laser intensity in W/cm^2 
# set density scale length in m (varies between 300e-6 - 100e-6 in the report) 
dens_scale_len = 500 * micron
# set the number of particles per cell (set to 2048 in the report)
# set to 100 to save time
ppc = 100

# For this example input deck, the number of timesteps and grid 
# cells are fixed at 4001 and 6473 respectively (see in example_input_deck)
nx = 6473
timesteps = 4001

t_end = 2.0 * pico

### Create sub-directories

In [ ]:
for i in range(len(sub_dirs)):
    epoch_sim_sub_dir(dir = dir, sub_dir= sub_dirs[i], input_file = input_file, I = intensity, Ln = dens_scale_len, ppc = ppc)

### Run all sub-directories at the same time using 2 processors each (20 in total)

In [ ]:
def run_epoch_in_parrallel():
    pool = Pool(processes=len(dirs))
    pool.map(run_epoch, dirs)

if __name__ == '__main__':
    run_epoch_in_parrallel()

# Extracting the Reflectivity - $\mathcal{P}$

The reflectivity ($\mathcal{P}$) is defined as the ratio of the average back-scattered electromagnetic wave intensity to that of the intialised laser intensity:

$$
\mathcal{P} = \frac{\langle I_{bs} \rangle}{I_{L}} .
$$

The energy flux (i.e intensity) of a electromagnetic wave can be found from the Poynting vector ($\mathbf{S}$), defined by the electric and magnetic fields of the electromagnetic wave:

$$
\mathbf{S} = \frac{1}{\mu_0} \mathbf{E} \times \mathbf{B}.
$$

The flux leaving in the $\mathbf{x}$ direction is therefore given by:
$$
S_{x} = \mathbf{S} \cdot \mathbf{\hat{x}}  = \frac{1}{\mu_0} (E_{y}B_{z} -{E_{z}B_{y}}) \approx \frac{1}{\mu_0} E_{y}B_{z},
$$
where the $E_z$ and $B_y$ fields being negliable as the backscattered light has the same polarisation as the laser (in this case the laser is polarised in the $y$-direction).

### Read in Electric and Magnetic fields from all data files (timesteps) 

In [ ]:
def get_2D_Ey_field(dir):
        # create space-time electric field array
        Ey = np.zeros((nx, timesteps))
        for i in range(timesteps):
            fname = f'{dir}/fields_'+str(i).zfill(4)+'.sdf'
            data = sdf.read(fname, dict = True)
            Ey[:, i] = data['Electric Field/Ey'].data
        return Ey
        
def get_2D_Bz_field(dir):
        # create space-time electric field array
        Bz = np.zeros((nx, timesteps))
        for i in range(timesteps):
            fname = f'{dir}/fields_'+str(i).zfill(4)+'.sdf'
            data = sdf.read(fname, dict = True)
            Bz[:, i] = data['Magnetic Field/Bz'].data
        return Bz

## Frequency Filtering 

Before we begin to implement the poynting flux formula, we need to be able to extract the electric and magnetic field contributions corresponding to the backscattered light. In 1D this is simplified as we only have two signals, one corresponding to the laser and the other to the backscattered light.

The laser signal is well-defined (as we know the frequncey of the laser), so we aim to extract this signal from each field using a windowed sinc filter so that we can seperate the frequncies hidden within the outputted field signal.
##### (Windowed sinc filter reference : http://www.dspguide.com/ch16/2.htm (Equation 16-4))

A bandpass filter was constructed by convolving low-pass (lpf) and high-pass filter (hpf), whcih are defined in frequncey space as:

$$
A_{\text{lpf}} = \begin{cases}
    1 ,& \text{if } \omega \leq \omega_c\\
    0,              & \omega > \omega_c
\end{cases}
\,\,
;
\,\,
A_{\text{hpf}} = \begin{cases}
    1 ,& \text{if } \omega \geq \omega_c\\
    0,              & \omega < \omega_c
\end{cases}
$$
where $\omega_{c}$ is some cut-off frequency. In real space, the low-pass and high-pass filters are mathematically defined as:

1. #### Sinc-filter
$$
 h_{i}\left(\omega\right) = \underbrace{\frac{\mathrm{sin}\left(\frac{2\pi\omega}{\omega_{N}} \left(i -M/2 \right)\right)}{\left(i - M/2\right)}}_\text{Sinc filter} \underbrace{\left[0.42 - 0.5 \text{cos}\left(\frac{2\pi i}{M}\right) + 0.08\text{cos}\left(\frac{4\pi i}{M}\right)\right]}_\text{Blackman Window}
$$

2. #### low-pass filter
$$
h_{i}^{\mathrm{lpf}} = \frac{h_{i}\left(\omega = \omega_{\mathrm{ub}}\right)}{\sum_{j} h_{j}\left(\omega = \omega_{\mathrm{ub}}\right)}, \,\, ; \,\, \omega_{\mathrm{ub}} = 1.15 \omega_L
$$

2. #### high-pass filter
$$
h_{i}^{\mathrm{hpf}} = - \frac{h_{i}\left(\omega = \omega_{\mathrm{lb}}\right)}{\sum_{j} h_{j}\left(\omega = \omega_{\mathrm{lb}}\right)}, \,\, ; \,\, \omega_{\mathrm{lb}} = 0.85 \omega_L
$$

3. #### complete band-pass filter
$$
h^{\text{bpf}} = h^{\text{lpf}} * h^{\text{hpf}}
$$


If the truncation is too abrupt, we may introduce ripples into the passband, which leads to desirable frequencies not being passed with their correct amplitudes, as well as ripples in the stop band, meaning that undesirable signals will be unevenly attenuated. This is why the filter kernel is convolved with a Black-man window to help smooth out the frequency response.

Applying this band-pass filter to the $E_y (x,t)$ and $B_z (x,t)$ signals attenuates all frequencies outside of the range $0.85 \omega_L \leq \omega \leq 1.15 \omega_L$, which represents the transmitted laser light.


### Band-pass filter construction


In [ ]:
def winsincFIR(omega_c,omega_s,M):
    # cutoff frequency shoudl be a fraction of sampling frequency
    ker = np.sinc((omega_c / omega_s) * (np.arange(M) - (M - 1)/2))
    # Blackman window used for smooting filter
    ker *= np.blackman(M)
    # unit gain at zero frequency (normalisation)
    ker /= np.sum(ker) 
    return ker

def bandpass(w0,bw,omega_s,M):
    # Angular frequency used for NIF Laser
    omega = 5.36652868179e+15
    w0 = w0 * omega
    bw = bw * omega
    # upper and lower bound frequencies of bandpass
    ub = w0 + (bw / 2)
    lb = w0 - (bw / 2)
    # create high-pass filter with cutoff at the lower-bound
    # inverse low-pass filter
    hhpf = -1 * winsincFIR(lb,omega_s,M) 
    hhpf[(M - 1) // 2] += 1
    # create low-pass filter with cutoff at the upper-bound
    hlpf = winsincFIR(ub,omega_s,M)
    # convolve the two into a band-pass filter
    h = np.convolve(hlpf, hhpf)
    return h


The laser signal is simply extracted by convolving the chosen field with the band-pass filter:


$$
E^{\text{Laser}}_y(x,t) = E_y(x,t) * h^{\text{bpf}}
$$
$$
B^{\text{Laser}}_z(x,t) = B_z(x,t) * h^{\text{bpf}}
$$

with the backscatterd signals being extracted from simple subtraction of this laser signal from the original:

$$
E^{\text{Back-scatter}}_y(x,t) = E_y(x,t) - E^{\text{Laser}}_y(x,t)
$$
$$
B^{\text{Back-scatter}}_z(x,t) = B_z(x,t) - B^{\text{Laser}}_z(x,t) 
$$

### Extract back-sacttered signals

In [ ]:
def get_filtered_signals(dir, laser = False):
        # required fields
        Ey = get_2D_Ey_field(dir) # Ey(x,t) field
        Bz = get_2D_Bz_field(dir) # Bz(x,t) field

        n,m = Ey.shape # array size
        omega_0 = 1.0 # normalised laser frequency 
        omega_bw = 0.3 # bandswidth centred at laser frequency
        T_end = t_end # sim end time
        N = timesteps # number of time steps
        dt = T_end/N # time step
        omegaNyq = pi/dt # Nyquist Frequency
        omega_s = 2*pi/dt # sampling frequency 
        M = 1001 # half length of the filter kernel (must be odd) 

        h = bandpass(omega_0,omega_bw,omegaNyq,M) #bandpass filter

       
        # Laser signals
        Ey_laser = np.zeros((n, m))
        Bz_laser = np.zeros((n, m))

        # SRS signals
        Ey_SRS = np.zeros((n, m))
        Bz_SRS = np.zeros((n, m))

        # Fill arrays with data
        for i in range(n):
            # laser signals
            Ey_laser[i, :] = np.convolve(Ey[i,:],h,mode='same')
            Bz_laser[i, :] = np.convolve(Bz[i,:],h,mode='same')
            # SRS signals
            Ey_SRS[i, :] = Ey[i,:] - Ey_laser[i,:]
            Bz_SRS[i, :] = Bz[i,:] - Bz_laser[i,:]

        if laser:    
            return Ey_laser, Bz_laser
        else:    
            return Ey_SRS, Bz_SRS


Now that we have the extracted signals, we can estimate the backsacttered energy flux. The electric and magnetic fields are recorded at many spatial locations over many time steps. So to estimate the total back-sactter intensity it becomes sensible to average over both time and space, such that:

$$
\langle I_{bs} \rangle = \frac{\sum_{i = 1}^{N_{x}} \sum_{j = 1}^{N_{t}} E^{\mathrm{Back-scatter}}_{y}\left(x_i, t_j\right)B^{\mathrm{Back-scatter}}_{z}\left(x_i, t_j\right)}{N_x N_t \mu_0}
$$

where $N_t$ and $N_x$ equate to the number of timesteps and grid cells in which the averaging is performed over.
The time averaging is performed over all time, where as the spatial averaging is done over cells close to the left-hand (laser entry) boundary.


Thus, $N_{x}$ and $N_{t}$ are taken to be 10 and 4001 respectively.

(At low intensities, the recorded poynting flux is domianted by noise as we have little to no back-scattered light being generated, in which the poynting vector is found to fluctuate between postive and negative values over time. As we are only considering light that is scattered backwards, we only want to average over the negative poytning vectors in this case, which is done by simply setting any positive poynting vectors to zero when averaging.)

### Extract reflectivity - $\mathcal{P}$

In [ ]:
## N_x = 10
## N_t = 4001
def get_bsrs(dir, ncells = 10, refelctivity = True):
    # get required field signals
    Ey, Bz = get_filtered_signals(dir, laser = False)           
    W_cm2 = 1e4 # Convert to W_cm2
    factor = mu0*W_cm2 # Denominator of Sx
    S = Ey*Bz/factor # poynting flux
    # integrate/average over time at each grid point
    sum_t = np.zeros(nx)
    for i in range(timesteps):
        sig = S[:,i]
        indx = np.where(sig > 0) # only care for backward travelling flux
        sig[indx] = 0
        sum_t += sig
    S_t_av = np.abs(sum_t)/timesteps
    # for backward travelling signals, we want to average close to the left-hand boundary
    sum_x = 0
    for i in range(ncells):
        sum_x += S_t_av[i]
    S_av = sum_x/ncells
    if refelctivity:
        return S_av/intensity
    else:
        return S_av

### Extract $\mathcal{P}$ for each EPOCH run

In [ ]:
P_data = np.array([])
for dir in dirs:
    P = get_bsrs(dir, ncells = 10, refelctivity=True)
    print(f'result for dir = {dir}, P = {P}')
    P_data = np.append(P_data, P)

### Convergence of Mean and Variance

In [ ]:
def average(array, num):
    data = array[:num]
    n = len(data)
    av = 0
    for i in range(n):
        av += data[i]
    av /= n
    return av

def variance(array, num):
    mean = average(array, num)
    data = array[:num]
    n = len(data)
    sum_ = 0
    for i in range(n):
        sum_ += (data[i]-mean)**2
    var = sum_/n
    return var

In [ ]:
means = []
variances = []
N_samples = []
for i in range(len(P_data)):
    N_samples.append(i+1)
    means.append(average(P_data, num = i+1))
    variances.append(variance(P_data, num = i+1))
    
fig,ax = plt.subplots()
ax.plot(N_samples, means, '-o', color ='b')
ax.set_xlabel('Number of Sims')
ax.set_ylabel('Mean Reflectivity')
ax2.set_ylabel('Variance of Reflectivity')
ax2=ax.twinx()
ax2.plot(N_samples, variances,'-o', color = 'r')
ax2.set_ylabel('Variance of Reflectivity')
plt.gcf().set_size_inches(10,8)

### Mean reflectivity and error from ten EPOCH samples

In [ ]:
P_mean = np.mean(P_data)
P_var = np.var(P_data)
P_err = 2.0*np.sqrt(P_var)

print(f'Mean reflectivity = {P_mean} W/cm^2')
print(f'Varaiance of reflectivity = {P_var}')
print(f'Error in reflectivity = {P_err} W/cm^2')

## Gaussian Process Regression Model

Reflectivity Data was collected at many intensity and density scale length points, for which a Gaussian process model was trained on.

We assume that the underlying function $f(x)$, can be represnted as a vector $\mathbf{f}$, drawn from a multivariate normal distribution with zero mean and covariance matrix $\mathbf{K}$:
$$
f \sim \mathcal{GP} \rightarrow \mathbf{f} \sim \mathcal{N}(\mathbf{0}, \mathbf{K}).
$$
As we can see from the EPOCH simulations, we have noisy observations due to sum inherent noise from plasma fluctuations. However, this noise is not homoscedastic (i.e not the same at each intensity), but varies depending on the set intensity. Thus, we define our noise observations in the following way:

$$
 y_{i} = f\left(\bold{x}_{i}\right) + \epsilon_{i} \,\,;\,\,  \epsilon_{i} | \bold{x}_{i} \sim \mathcal{N}(0,\,\sigma_{n,i}^{2}),
$$
where $\sigma^2_{n,i}$ is the measured noise variance at point $i$.

Assuming additive independnece, the covariance matrix for the observations is augmented by this noise such that:
$$
\Sigma [\bold{y}] = K(X,X) + \text{diag}(\sigma^2_{n}(X)).
$$

### Noise GP Model 
To extract the estimated noise from plasma fluctuations at new locations, a further model was required for when we make predictions for the reflectivity at new points. For simplicity, this was done also using a Gaussian process, in which the noise model is trained on the recorded variances at each input space co-ordinate.

The noise GP is defined similarly to the reflectivity GP model excpet for two asepcts. Firstly, the kernel functions used to describe $K$ for both models differ, the main GP uses a $\textbf{Rational Quadratic}$ kernel, where as the noise model uses an $\textbf{Exponential}$ kernel, as the noise model is expected to be less smooth in nature.  

Secondly, the extracted variance observations are assumed to be exact as there is no way in which to sensibly quantify an error on the noise itself, so the covariance matrix for the noise observations is augmented by some small number, e.g:
$$
\Sigma_{noise} [\bold{y}] = K_{noise}(X_{noise},X_{noise}) + 1.0\times 10^{-6} \bold{I}.
$$



### Set Training data for both GP models

This example is for a 1D test case of 20 logarithmically spaced intensity points in the range of $4.0\times 10^{14} - 4.0\times 10^{15} \,\, W/cm^2$. This was also done at the same resoloution as the EPOCH samples performed earlier in this notebook (PPC = 100)

In [ ]:
# input data (I, Ln)
input_file = 'Training_data/train_inputs.json'
# mean refelectivity at each input 
output_file = 'Training_data/train_outputs_mean.json'
# recorded noise variance at each input
var_file = 'Training_data/train_outputs_var.json'
# fraction of data on which to train both GP's on
train_frac = 0.2

### Call GP class contaning all functions for GP regression

In [ ]:
gp = LPI_GP(input_file=input_file, output_file=output_file,\
            var_file=var_file, train_frac=train_frac)

### Set/fix the training data

In [ ]:
gp.set_training_data()

## Hyper-parameter optimisation

Both the rational quadratic and exponential kernels have two hyper-parameter values which need to be determined (a lengthscale $l$, and a vaiance $\sigma^{2}_{f}$).

To set these values, a simple grid search routine is used for a space of lengthscales and variances, in which at each point on the grid, the ngeative log-likelihood function is estimated. For an assumed Gaussian likelihood distribution, the the ngeative log-likelihood function for  set of hyper-paramters $\Theta = [l, \sigma^2_{f}]$ is given by:

$$
\mathcal{L}(\Theta) = \frac{1}{2}\bold{y}^T\left[\bold{K} + \bold{D}\right]^{-1}\bold{y} - \frac{1}{2}\mathrm{log}\left|\bold{K} + \bold{D}\right| - \frac{N}{2}\mathrm{log}{2\pi},
$$

where $K = K$ and $D = \text{diag}(\sigma^2_n(X))$ for our general GP model and $K = K_{noise}$ and $D = 1.0\times 10^{-6} \bold{I}$, for the noise model.

### Optimise noise hyper-parameters

In [ ]:
gp.optimise_noise_GP()

### Optimise reflectivity model hyper-parameters

In [ ]:
gp.optimise_GP()

### test train plots

In [ ]:
gp.test_train_plot()
plt.gcf().set_size_inches(40,15)

## Model Prediction

One can now make predictions at new locations $\bold{X^*} = [\bold{x^*}_1,.., \bold{x^*}_M]$ giving new function values $\bold{f^*} = [f(\bold{x^*})_1,..,f(\bold{x^*})_M]$ described via the Gaussian distribution:
$$
\begin{bmatrix}
\bold{y} \\
\\
\bold{f^*}  
\end{bmatrix}
\sim \mathcal{N}\left(\bold{0}, \begin{bmatrix}
\underbrace{\bold{K}(\bold{X}, \bold{X})}_{\bold{K}} + \bold{D} & \underbrace{\bold{K}(\bold{X}, \bold{X^*})}_{\bold{k_*}}\\
\\
\underbrace{\bold{K}(\bold{X^*}, \bold{X})}_{\bold{k_*}^T} & \underbrace{\bold{K}(\bold{X^*}, \bold{X^*}}_{\bold{K^*}})  
\end{bmatrix}\right).
$$
Finally, applying this result to obtain the conditional distribution $\bold{f^*}|\bold{y}$, we find that the posterior distribution for $\bold{f^*}$ is a Gaussian with mean and covariance given by:
$$
\mathbb{E}[\bold{f^*}] = \bold{k_*}^T\left[\bold{K} + \bold{D}\right]^{-1}\bold{y}
$$
$$
\Sigma[\bold{f^*}] = \bold{K^*} - \bold{k_*}^T\left[\bold{K} + \bold{D}\right]^{-1} \bold{k_*}.
$$

To account for the heteroscedastic noise nature of the
problem, the covariance matrix must be augmented to
include this noise as done earlier:
$$
\Sigma[\bold{f^*}] = \bold{K^*} - \bold{k_*}^T\left[\bold{K} + \bold{D}\right]^{-1} \bold{k_*} + D^* \,\, \text{where} \,\, D^* =  \text{diag}(\sigma^2_n(X^*)).
$$

Here, the estimation for the noise variance at the unkown loaction ($D^*$) is estimated from the noise GP model such that:
$$
\sigma^{2}_n(X^*_i) = \bold{k^{noise}_*}^T\left[\bold{K_{noise}} + \bold{D_{noise}}\right]^{-1}\bold{y}
$$
which is equivalent to the mean of the posterior distribution from the noise GP model.

----------------------------------------------------------------------------------------------------
Note: To avoid having to invert any matricies, Cholesky decomposition was used such that the weights $\alpha = \left[\bold{K} + \bold{D}\right]^{-1}y$, can be found in the following way:

$$
LL^{T} = \left[\bold{K} + \bold{D}\right] \rightarrow \left[\bold{K} + \bold{D}\right]\alpha = y \rightarrow LL^T \alpha = y
$$
$$
\alpha = L^T/(L/y)
$$


### Show Prediction over whole training region

In [ ]:
X_star = np.geomspace(4e14, 4e15, 100)[:,None]
Y_star, V_epi, V_noise = gp.GP_predict(X_star, get_var=True)

In [ ]:
# Inputs and mean ouputs that the reflectivity model is trained on 
X = np.exp(gp.get_input())
Y = np.exp(gp.get_output())

# all sample points (all 10 samples of each intensity point (20x10))
X_all = np.exp(read_json_file('Training_data/all_inputs.json'))
Y_all = np.exp(read_json_file('Training_data/all_outputs.json'))


In [ ]:
plt.rcParams["figure.figsize"] = [14, 10]

error_epi = 2.0*np.sqrt(V_epi)
error_tot = 2.0*np.sqrt(V_epi + V_noise)

Y_s = Y_star.flatten()
X_s = X_star.flatten()

plt.loglog(X_s, Y_s, color = 'blue', label = 'GP Mean')
plt.fill_between(X_s, (Y_s-error_epi), (Y_s+error_epi), alpha = 0.3, color = 'cyan', label = 'Epistemic Error')
plt.fill_between(X_s, (Y_s-error_tot), (Y_s+error_tot), alpha = 0.15, color = 'red', label = 'Total Error')
plt.plot(X_all, Y_all, 'kx', color = 'red', label = 'All Samples', alpha = 0.8)
plt.plot(X, Y, 'kx', color = 'blue', label = 'Mean Samples')
plt.xlim(4e14, 4e15)
plt.ylim(2e-3, 2e-1)

plt.ylabel(r'Reflectivity - $\mathcal{P}$')
plt.xlabel(r'$I_{L} \,\, W/cm^{2}$')
plt.legend(loc = 0)

## Error Bar validation

Here we comapre the extracted error bars from the GP model with the recorded error from our EPOCH sampled at the same intensity.

In [ ]:
Y_star, V_epi, V_noise = gp.GP_predict(X_star = np.array([intensity])[:,None], get_var=True)
error_epi = 2.0*np.sqrt(V_epi)
error_noise = 2.0*np.sqrt(V_noise)
error_tot = 2.0*np.sqrt(V_epi + V_noise)

In [ ]:
err_compare = [P_err, error_epi, error_noise, error_tot]
# plot scaled sensitivities
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(['Samaple Error', 'GP Epistemic Error', 'GP Noise Error', 'GP Total Error'], err_compare, 'o')
ax.set_xticklabels(['Samaple Error', 'GP Epistemic Error', 'GP Noise Error', 'GP Total Error'], rotation=90)
ax.set_ylabel(r'Error')